In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import pandas as pd

driver = webdriver.Chrome()
driver.maximize_window()
driver.get("https://www.instagram.com")

In [ ]:
# log in
username = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.NAME, "username"))
)
password = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.NAME, "password"))
)
login = driver.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[3]/button/div')

ID = "your_username"
username.clear()
password.clear()
username.send_keys(ID)
password.send_keys("your_password")

login.click()
# wait until search box appears
wait = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input'))
)

In [ ]:
# crawl urls and descriptions of saved posts
driver.get("https://www.instagram.com/" + ID + "/saved/all-posts/")
time.sleep(3)

links = []
descriptions = []

last_height = driver.execute_script("return document.body.scrollHeight")
last_link = ""

while True:
    # add links and descriptions
    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.find("article")
    posts = articles.find_all("div", "_aabd _aa8k _aanf")
    start_appending = False

    for post in posts:
        info = post.find("a", "qi72231t nu7423ey n3hqoq4p r86q59rh b3qcqh3k fq87ekyn bdao358l fsf7x5fv rse6dlih s5oniofx m8h3af8h l7ghb35v kjdc1dyq kmwttqpk srn514ro oxkhqvkx rl78xhln nch0832m cr00lzj9 rn8ck1ys s3jn8y49 icdlwmnq _a6hd")
        link = info.attrs['href']
        desc = info.find("img", "_aagt")
        
        if last_link == "":
            start_appending = True

        if start_appending:
            links.append("https://www.instagram.com" + link)
            last_link = link
            try:
                alt = desc.attrs['alt']
                descriptions.append(alt)
            except:
                descriptions.append("-")
        elif link == last_link:
            start_appending = True

    # scroll and load
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(7)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [ ]:
# display as data frame
pd.set_option("display.max_rows", None)

data = {'url': links, 'description': descriptions}
data_frame = pd.DataFrame(data)

data_frame